In [1]:
#https://www.frontiersin.org/articles/10.3389/fnhum.2020.00338/full
#!pip install tensorflow==2.9.1
import tensorflow as tf
print(tf.__version__)
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
import pywt
import numpy as np
import imageio
import tensorflow as tf
import cv2
import time
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from tensorflow.keras.models import Model, load_model
from sklearn.metrics import classification_report
import os
from scipy import signal
from sklearn import preprocessing
print(tf.__version__)

2.12.0
2.12.0


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')

for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)

In [4]:
data = pd.read_csv('/content/drive/MyDrive/TCC-Physionet/dadosPhysionet.csv', delimiter=";")


In [5]:
eletrodes = ['F3', 'F1', 'FZ', 'F2', 'F4', 'FC5', 'FC3', 'FC1', 'FCZ',
             'FC2', 'FC4', 'FC6', 'C5', 'C3', 'C1', 'CZ', 'C2', 'C4',
             'C6', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'CP6',
             'O1', 'O2']
labels = {'1': 'rest', '2': 'left', '3': 'right'}
selected_eletrodes = ['FC3', 'FC4']

In [9]:

widths = np.arange(1, 51)
labels = []
signals = []
pacients = data["paciente"].unique()
for pacient in pacients:
    measures = data.loc[data['paciente'] == pacient]['medicao'].unique()
    for measure in measures:
        label = data.loc[(data['paciente'] == pacient) & (data['medicao'] == measure)]['label'].unique()[0]
        signal_list = []
        for eletrode in selected_eletrodes:
            sign = data.loc[(data['paciente'] == pacient) & (data['medicao'] == measure) & (data['eletrodo'] == eletrode)].iloc[:,4:].to_numpy().reshape(640)
            signal_list.append(sign)

        img = np.zeros([2,640])
        for i in range(len(selected_eletrodes)):
            img[i] = signal_list[i]
        img = zip(img).tonumpy()
        len(img)
        len(img[0])
        signals.append(img)
        if(label == 1):
          labels.append(0)
        elif(label == 2):
          labels.append(1)
        else:
          labels.append(2)
        input()


AttributeError: ignored

In [ ]:
print(len(labels))

3147


In [ ]:
signals_train = []
labels_train = []

signals_test = []
labels_test = []

signals_val = []
labels_val = []

for i in range(len(signals)):
  if(i%3 == 0):
    if(len(signals_train) < 2203):
      signals_train.append(signals[i])
      labels_train.append(labels[i])
    elif(len(signals_test) < 472):
      signals_test.append(signals[i])
      labels_test.append(labels[i])
    else:
      signals_val.append(signals[i])
      labels_val.append(labels[i])

  if(i%3 == 1):
    if(len(signals_test) < 472):
      signals_test.append(signals[i])
      labels_test.append(labels[i])
    elif(len(signals_val) < 472):
      signals_val.append(signals[i])
      labels_val.append(labels[i])
    else:
      signals_train.append(signals[i])
      labels_train.append(labels[i])

  if(i%3 == 2):
    if(len(signals_val) < 472):
      signals_val.append(signals[i])
      labels_val.append(labels[i])
    elif(len(signals_test) < 472):
      signals_test.append(signals[i])
      labels_test.append(labels[i])
    else:
      signals_train.append(signals[i])
      labels_train.append(labels[i])

In [ ]:
img_generator = tf.keras.preprocessing.image.ImageDataGenerator()
features_train = img_generator.flow(
    x=np.array(signals_train).reshape(2203, 2, 640,1),
    y=np.array(labels_train).reshape(2203,1),
    batch_size=32,
    shuffle=True)
features_test = img_generator.flow(
    x=np.array(signals_test).reshape(472, 2, 640,1),
    y=np.array(labels_test).reshape(472,1),
    batch_size=32,
    shuffle=True)
features_val = img_generator.flow(
    x=np.array(signals_val).reshape(472, 2, 640,1),
    y=np.array(labels_val).reshape(472,1),
    batch_size=32,
    shuffle=True)


In [ ]:
model = tf.keras.Sequential(
    [
    tf.keras.layers.Conv2D(25, (1,11), padding='valid', activation="relu",input_shape=(2, 640, 1)),
    tf.keras.layers.SpatialDropout2D(0.5),
    tf.keras.layers.Conv2D(25, (2,1), padding='valid'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D((1, 2), strides=3),
    tf.keras.layers.Conv2D(50, (1,11), padding='valid', activation="relu"),
    tf.keras.layers.SpatialDropout2D(0.5),
    tf.keras.layers.MaxPooling2D((1, 4), strides=3),
    tf.keras.layers.Conv2D(100, (1,11), padding='valid'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.SpatialDropout2D(0.5),
    tf.keras.layers.MaxPooling2D((1, 4), strides=3),
    tf.keras.layers.Conv2D(200, (1,11), padding='valid'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D((1, 2), strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(3, activation="softmax")
]
)

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 2, 630, 25)        300       
                                                                 
 spatial_dropout2d_3 (Spatia  (None, 2, 630, 25)       0         
 lDropout2D)                                                     
                                                                 
 conv2d_6 (Conv2D)           (None, 1, 630, 25)        1275      
                                                                 
 batch_normalization_3 (Batc  (None, 1, 630, 25)       100       
 hNormalization)                                                 
                                                                 
 activation_3 (Activation)   (None, 1, 630, 25)        0         
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 1, 210, 25)      

In [ ]:
epochs=100

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(features_train, epochs=epochs, validation_data=(features_test))

Epoch 1/100
69/69 [==============================] - 5s 13ms/step - loss: 2.5931 - accuracy: 0.3600 - val_loss: 2.3770 - val_accuracy: 0.0000e+00
Epoch 2/100
69/69 [==============================] - 1s 8ms/step - loss: 2.4914 - accuracy: 0.3845 - val_loss: 2.3779 - val_accuracy: 0.0000e+00
Epoch 3/100
69/69 [==============================] - 1s 8ms/step - loss: 2.4630 - accuracy: 0.3291 - val_loss: 2.3746 - val_accuracy: 1.0000
Epoch 4/100
69/69 [==============================] - 1s 8ms/step - loss: 2.4475 - accuracy: 0.4330 - val_loss: 2.3743 - val_accuracy: 0.0000e+00
Epoch 5/100
69/69 [==============================] - 1s 8ms/step - loss: 2.4444 - accuracy: 0.5502 - val_loss: 2.3743 - val_accuracy: 0.0000e+00
Epoch 6/100
69/69 [==============================] - 1s 8ms/step - loss: 2.4442 - accuracy: 0.5452 - val_loss: 2.3750 - val_accuracy: 0.0000e+00
Epoch 7/100
69/69 [==============================] - 1s 8ms/step - loss: 2.4440 - accuracy: 0.5238 - val_loss: 2.3743 - val_accuracy:

In [ ]:
label = features_val.y[3]
res = model.predict(features_val.x[3].reshape(1,2,640,1))

1/1 [==============================] - 0s 154ms/step


In [ ]:
print(label)
print(res)

[1]
[[0.33856013 0.32942805 0.3320118 ]]


In [ ]:
[0]
[[0.33594087 0.33332655 0.33073258]]

In [ ]:
cont_0 = 0
cont_1 = 0
cont_2 = 0

for i in range(len(features_test.y)):
  if(features_test.y[i] == 0):
    cont_0 += 1
  elif(features_test.y[i] == 1):
    cont_1 += 1
  elif(features_test.y[i] == 2):
    cont_2 += 1

print(cont_0, cont_1, cont_2)


244 116 112


472

In [ ]:
https://stackoverflow.com/questions/66714485/how-can-i-train-my-cnn-model-on-dataset-from-a-csv-file
https://towardsdatascience.com/data-preprocessing-and-network-building-in-cnn-15624ef3a28b
https://towardsdatascience.com/a-guide-to-an-efficient-way-to-build-neural-network-architectures-part-ii-hyper-parameter-42efca01e5d7